# Datan analysointi K2020_2 - Tehtävät kerta 3

Palauta Optiman palautuslaatikkoon viimeistään su 16.2. Kirjoita palautuslaatikon kommenttikenttään, mitä tehtävät olet tehnyt kokonaan/osittain.

**Tee tehtävät ilman for-silmukoita tms.**

**Opiskelijan nimi:**


## 1. Maakunnat





Lue DataFrameen tiedosto jossa on tilastotietoja Suomen kunnnista (vuosilta 2017-19). Kenttäerotin puolipiste ja deimaalierotin pilkku.

https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/kunnat.txt


a) Laske kuinka paljon maapinta-alaa on kussakin seutukunnassa ja tulosta maapinta-alaltaan 5 suurinta seutukuntaa suuruusjärjestyksessä.

b) Laske kuinka monta prosenttia kunkin maakunnan väestöstä asuu kaupungeissa (kuntamuoto) ja tulosta 5 suurinta %-osuutta. 
 
c) Laske ruotsinkielisten osuus maakunnittain (huom. et voi laskea prosenttiluvuista keskiarvoa, koska kuntien väkiluvut vaihtelevat). Tulosta 5 suurinta osuutta.




In [2]:
import pandas as pd
df = pd.read_csv('https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/kunnat.txt', sep=';', decimal=',')
#a.
print(df.groupby('seutukunta')['maapinta-ala'].sum().sort_values(ascending=False).head(5))
print(80*"-")
#b.
print((df.groupby('maakunta')['kuntamuoto'].apply(lambda x: x[x.str.contains('Kaupunki')].count()) / df.groupby('maakunta')['kuntamuoto'].size() * 100).round(0).sort_values(ascending = False).head(5))
print(80*"-")
#c.
df['ruotsi'] = df['Ruotsinkielisten osuus%'] / 100 * df['Väkiluku']
print((df.groupby('maakunta')['ruotsi'].sum() / df.groupby('maakunta')['Väkiluku'].sum().round(0)))

seutukunta
Pohjois-Lapin seutukunta    31893.91
Itä-Lapin seutukunta        20548.92
Tunturi-Lapin seutukunta    20512.50
Kehys-Kainuun seutukunta    13959.50
Rovaniemen seutukunta       11035.13
Name: maapinta-ala, dtype: float64
--------------------------------------------------------------------------------
maakunta
Pirkanmaa          55.0
Uusimaa            50.0
Etelä-Pohjanmaa    47.0
Kymenlaakso        43.0
Pohjois-Karjala    42.0
Name: kuntamuoto, dtype: float64
--------------------------------------------------------------------------------
maakunta
Ahvenanmaa           0.867708
Etelä-Karjala        0.001751
Etelä-Pohjanmaa      0.002925
Etelä-Savo           0.001724
Kainuu               0.001160
Kanta-Häme           0.003753
Keski-Pohjanmaa      0.090410
Keski-Suomi          0.001575
Kymenlaakso          0.007824
Lappi                0.002429
Pirkanmaa            0.003563
Pohjanmaa            0.494596
Pohjois-Karjala      0.000907
Pohjois-Pohjanmaa    0.001815
Pohjois-Savo    

## 2.  access.log

Lue erään palvelimen (3 tunnin ajan) access.log-tiedosto (15 Mt zip, noin 600 000 riviä) DataFrameen  
http://gpspekka.kapsi.fi/accesslog2.zip  
2-kerran teoriamateriaalissa on ohjeita datan lukemiseen.

Kentät  
* ip-osoite
* ident-id (-)
* userid (-)
* time
* request
* status
* size
* referer
* user agent


Parsi aikakentästä tunti ja minuutit ja laske jokaiselle minuutille kuinka monta eri ip-osoitetta on kyseisenä päivänä sivulla vieraillut.   

Tulosta listaa suuruusjärjestyksessä eli eniten eri ip-osoitteita sisältänyt minuutti ensin. 

Alkua:
```
12:55    792
12:51    789
12:54    789
12:53    787
12:56    786
12:52    783
14:00    773
14:01    769
13:59    762
12:50    760

```


In [117]:
df = pd.read_table('http://gpspekka.kapsi.fi/accesslog2.zip',
                header=None,
                sep = ' ',
                quotechar = '"',
                usecols = [0,1,2,3,4,5,6,7,8,9],
                names = ['ip-osoite', 'ident-id', 'userid', 'date', 'time', 'request', 'status', 'size', 'referer', 'user agent'])
df['time'] = df['date'] + df['time']
df = df.drop(columns=['date'])

df2['time'] = df['time'].str.split(':').str.get(1) + ':' + df['time'].str.split(':').str.get(2)
df3 = (df['ip-osoite'].groupby(df2['time']).nunique().nlargest(10))
print(df3)

time
12:55    792
12:51    789
12:54    789
12:53    787
12:56    786
12:52    783
14:00    773
14:01    769
13:59    762
12:50    760
Name: ip-osoite, dtype: int64


## 3. Kone

Tiedostossa http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/kone.csv  (11 Mt) on erään koneen toiminnasta dataa 10 sekunnin välein. Aikaleiman lisäksi datassa on sarake `katko`, joka on 1 kun koneen toiminta on ollut katkoksissa ja 0 kun kone on ollut toiminnassa.

Poimi datasta tietoa katkoista eli **peräkkäisistä 1-riveistä**. Listaa kustakin katkosta:  
  * monesko katko on kyseessä
  * koska katko alkoi (eli ensimmäinen aikaleima)
  * koska katko loppui (eli viimeinen aikaleima)
  * kauanko katko kesti tunteina (rivien määrä jaettuna 360:lla).
  
10 ensimmäistä katkoa pitäisi näyttää tältä:
![katkot](https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/kone.png)  

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/kone.csv')
df4 = pd.DataFrame(columns=['katko_nro'])
df.loc[df['katko'] > df.shift(1)['katko'], 'Muutos'] = 'Alkaa'
df.loc[df['katko'] > df.shift(-1)['katko'], 'Muutos'] = 'Päättyy'
df['Muutos'].fillna(0,inplace=True)

df2 = df.loc[df['Muutos'] == 'Alkaa']
df3 = df.loc[df['Muutos'] == 'Päättyy']
df2.reset_index(drop=True, inplace=True)
df3.reset_index(drop=True, inplace=True)

df4['alkoi'] = df2['aika']
df4['loppui'] = df3['aika']
df4['katko_nro'] = np.arange(len(df4))
df4 = df4.set_index('katko_nro')

df4['tunti'] = df4['alkoi'].str.split('T').str.get(1).str.split('.').str.get(0)
df4['tunti2'] = df4['loppui'].str.split('T').str.get(1).str.split('.').str.get(0)
df4['tunti'] = pd.to_timedelta(df4.tunti)
df4['tunti2'] = pd.to_timedelta(df4.tunti2)

df4['kesto'] = df4['tunti2'] - df4['tunti']
df4 = df4.drop(columns=['tunti', 'tunti2'])

print(df4.head(10))

                                 alkoi                       loppui    kesto
katko_nro                                                                   
0          2019-03-18T11:26:30.000000Z  2019-03-18T11:34:30.000000Z 00:08:00
1          2019-03-18T11:35:30.000000Z  2019-03-18T11:43:00.000000Z 00:07:30
2          2019-03-18T14:05:30.000000Z  2019-03-18T14:16:50.000000Z 00:11:20
3          2019-03-18T21:49:00.000000Z  2019-03-18T21:56:00.000000Z 00:07:00
4          2019-03-18T22:51:20.000000Z  2019-03-18T22:56:20.000000Z 00:05:00
5          2019-03-18T22:57:20.000000Z  2019-03-18T22:59:40.000000Z 00:02:20
6          2019-03-19T01:47:40.000000Z  2019-03-19T02:05:30.000000Z 00:17:50
7          2019-03-19T02:54:40.000000Z  2019-03-19T02:58:40.000000Z 00:04:00
8          2019-03-19T03:54:00.000000Z  2019-03-19T04:00:00.000000Z 00:06:00
9          2019-03-19T04:09:10.000000Z  2019-03-19T04:20:40.000000Z 00:11:30


## 4. Lumeton alkutalvi?

Lataa DataFrameen csv-tiedosto  
http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/saajkl20200118.csv

Tiedostossa on päivittäisiä säähavaintoja Jyväskylän lentoasemalta vuodesta 1959.

Laske tietojen perusteella jokaiselle "alkutalvelle" 1. marraskuuta - 18. tammikuuta  **"lumisumma"** jossa on alkutalven jokaisen vuorokauden lumensyvyys-lukema laskettu yhteen (ei ole mikään oikea meteorologinen termi).

Huomioita:
* lumisumma pitää laske talvittain, ei siis vuosittain. Tee uusi sarake joka kertoo mihin (alku)talveen kyseinen vuorokausi kuuluu. Siis tammikuu 2020 kuuluu samaan alkutalveen kuin joulukuu 2019.

* Ilmatieteen laitos käyttää arvoa -1 lumensyvyyden arvona jos "lunta ei ole näkyvissäkään". Ei haluta kuitenkaan miinustaa yhtä näinä päivinä, joten korvaa -1:t nollilla.

* Lisäksi joinain päivinä lumensyvyys todella (jostain syystä) puuttuu. Jos nämä jätetään laskusta pois, kyseisen talven lukema jää liian pieneksi (jos lunta on kuitenkin ollut). Lienee turvallista olettaa että lunta on varmaan ollut suunnilleen saman verran kuin edellisenä päivänä, joten täytä NaN-arvot edellsillä arvoilla

Anna tuloksena DataFrame, jossa on indeksinä talvi (joko muodossa 2019 tai 2019-2020) ja sarakkeina 
* alkutalventalven lumisumma 
* sijoitus lumisummien joukossa (lumisin alkutalvi = 1) 
* alkutalven lumipäivien lukumäärä (eli kuinka monena päivänä lumensyvyys on ollut suurempi kuin nolla)
* alkutalven suurin lumensyvyys.

mallia alusta ja lopusta:  
 ```
           Lumensyvyys  sija  lumipaivia  maksimi
Talvi                                            
1959-1960         1292    20          61       38
1960-1961         1305    19          61       52
1961-1962         1116    28          54       41
1962-1963         1065    30          65       35
1963-1964          850    41          69       22
           Lumensyvyys  sija  lumipaivia  maksimi
Talvi                                            
2015-2016          139    61          31       14
2016-2017          737    50          69       18
2017-2018         1476    15          62       49
2018-2019          568    53          39       31
2019-2020          856    39          68       30
 ```


In [4]:
df = pd.read_csv('http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/saajkl20200118.csv')
df.loc[df.Lumensyvyys < 0, 'Lumensyvyys'] = 0
df['Lumensyvyys'].fillna(method='ffill', inplace=True)

def talvi(row):
    if row['Kk'] >= 11:
        val = (row['Vuosi'])
    elif (row['Kk'] == 1) & (row['Pv'] < 19):
        val = ((row['Vuosi'] - 1))
    else:
        val = " "
    return val

df['Talvi'] = df.apply(talvi, axis=1)
df['Lunta'] = df['Lumensyvyys'].map(lambda x: 1 if x != 0 else 0)
#print(df.head(10))

dftulos = pd.DataFrame(df.groupby(df['Talvi'])['Lumensyvyys'].sum())
dftulos = dftulos[:-1]

dftulos['Sija'] = dftulos['Lumensyvyys'].rank(ascending = False)
dftulos['Lumipaivia'] = df.groupby('Talvi')['Lunta'].sum()
dftulos['Maksimi'] = df.groupby('Talvi')['Lumensyvyys'].max()
print(dftulos.head(10))

       Lumensyvyys  Sija  Lumipaivia  Maksimi
Talvi                                        
1959        1292.0  20.0          61     38.0
1960        1305.0  19.0          61     52.0
1961        1116.0  28.0          54     41.0
1962        1065.0  30.0          65     35.0
1963         850.0  41.0          69     22.0
1964        1174.0  25.0          62     36.0
1965        2109.0   5.0          73     58.0
1966        1266.0  23.0          57     54.0
1967        1047.0  32.0          53     29.0
1968        1693.0  10.0          79     45.0


## 5. Liiga

Tiedostossa http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/liiga.txt on SM-liigan (nyk. Liiga) runkosarjan ottelutukset 2010-luvulta.  

huom. -sarakkeessa on merkintä JA tai VL, jos ottelu on ratkennut vasta jatkoajalla tai voittomaalikilpailussa.

Voittanut (enemmän maaleja tehnyt joukkue) saa 3 pistettä ja hävinnyt 0 pistettä, paitsi jos ottelu on ratkennut vasta jatkoajalla tai voittomaalikilpailussa, voittanut joukkue saa 2 pistettä ja hävinnyt yhden pisteen.



Generoi tulosten perusteella "vuosikymmenen sarjataulukko", jossa on kullekin joukkueelle omana sarakkeenaan
* sijoitus
* joukkueen nimi
* pelatut ottelut
* 3 pisteen voitot
* 2 pisteen voitot (JA/VL)
* 1 pisteen tappiot (JA/VL)
* 0 pisteen tappiot
* tehdyt maalit ja päästetyt maalit (samassa sarakkeessa)
* pisteet



Lajittele sarjataulukko pisteiden mukaan (eniten pisteitä ekana). Tasapisteissä olevat joukkueet menevät maalieron (tehdyt maalit - päästetyt maalit) mukaiseen järjestykseen, ja jos sekin on sama niin tehdyt maalit ratkaisevat.

tavoiteltu lopputulos:

```
    sija   joukkue  ott   v3  v2  t1   t0     maalit  pist
0      1    Kärpät  574  295  70  51  158  1678-1244  1076
1      2       JYP  575  277  62  58  178  1614-1311  1013
2      3   Tappara  575  261  78  62  174  1598-1387  1001
3      4      HIFK  575  264  62  63  186  1637-1403   979
4      5     KalPa  574  224  69  68  213  1414-1400   878
5      6     Lukko  574  226  62  67  219  1516-1479   869
6      7     Ässät  574  215  64  56  239  1428-1501   829
7      8  Pelicans  574  215  57  56  246  1530-1595   815
8      9       HPK  575  198  80  60  237  1416-1498   814
9     10     SaiPa  574  201  66  71  236  1451-1576   806
10    11       TPS  572  193  59  78  242  1410-1596   775
11    12     Ilves  574  192  55  60  267  1426-1643   746
12    13     Blues  360  126  43  36  155    852-943   500
13    14   Jokerit  240  102  29  37   72    672-586   401
14    15     Sport  334   87  37  47  163   810-1028   382
15    16    KooKoo  275   79  32  42  122    664-809   343
16    17   Jukurit  215   56  21  34  104    486-603   244
```

(jos saat Ässille 832 pistettä, ohjelmakoodisi tulkitsee voittajan tuloksesta 11-3 väärin...) 


         pvm         ottelu tulos huom   kotij  vierasj  kotim  vierasm  \
0  16.9.2010   HIFK-Jokerit   5-3    0    HIFK  Jokerit      5        3   
1  16.9.2010  Ilves-Tappara   1-5    0   Ilves  Tappara      1        5   
2  16.9.2010      KalPa-JYP   3-2   VL   KalPa      JYP      3        2   
3  16.9.2010   Kärpät-SaiPa   1-2    0  Kärpät    SaiPa      1        2   
4  16.9.2010    Lukko-Ässät   2-3    0   Lukko    Ässät      2        3   

   kotipist  vieraspist voittaja  
0         3           0     HIFK  
1         0           3  Tappara  
2         2           1    KalPa  
3         0           3    SaiPa  
4         0           3    Ässät  
--------------------------------------------------
kotij
Blues       321
HIFK        551
HPK         474
Ilves       410
JYP         579
Jokerit     222
Jukurit     129
KalPa       492
KooKoo      197
Kärpät      618
Lukko       497
Pelicans    467
SaiPa       456
Sport       209
TPS         439
Tappara     549
Ässät       450
Name: koti